### Data Loading and Merging

In [23]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora, models
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
nltk.download('punkt')
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
# ! pip install textblob
from textblob import TextBlob
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/giteshbibra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/giteshbibra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/giteshbibra/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [56]:
# Load the data files
jeep_avenger = pd.read_csv('CA-Jeep Avenger/comments_JeepAvenger.csv')
hyundai_kona = pd.read_csv('CA-Hyundai Kona/comments_HyundaiKona.csv')
peugeot_e2008 = pd.read_csv('CA-Peugeot e2008/comments_Peugeot_e2008.csv')
vauxhall_mokka=pd.read_csv('CA-Vauxhall Mokka/comments_Vauxhall_Mokka_EV.csv')
mini_cooper=pd.read_csv('CA-MiniCooper EV/comments_MiniCooper_EV.csv')


In [57]:
# jeep_avenger.head()
# hyundai_kona.head()
# peugeot_e2008.head()
# vauxhall_mokka.head()
mini_cooper.head()

,Timestamp,Username,VideoID,Comment,Date
0,2024-07-23T15:19:10Z,@raider7966,Las92RDdkOM,Sorry not sure what they were thinking with th...,2024-07-23T15:19:10Z
1,2024-07-15T16:47:45Z,@jackdaniels2905,Las92RDdkOM,Heating and cooling controls need to be physic...,2024-07-15T16:47:45Z
2,2024-06-20T21:51:48Z,@pqvid,Las92RDdkOM,You know how you can control the regen? With y...,2024-06-20T21:51:48Z
3,2024-06-19T19:54:27Z,@silverlover5817,Las92RDdkOM,Thanks for the video. All the videos I have se...,2024-06-19T19:54:27Z
4,2024-07-27T11:34:59Z,@Sc0oby26th,Las92RDdkOM,Err.....it's electric,2024-07-27T11:34:59Z


In [58]:
# Add a column for the car model in each dataset
jeep_avenger['Car'] = 'Jeep Avenger'
hyundai_kona['Car'] = 'Hyundai Kona'
peugeot_e2008['Car'] = 'Peugeot e2008'
vauxhall_mokka['Car'] = 'Vauxhall Mokka'
mini_cooper['Car'] = 'Mini Cooper'

In [59]:
# jeep_avenger.head()
# hyundai_kona.head()
# peugeot_e2008.head()
# vauxhall_mokka.head()
mini_cooper.head()

,Timestamp,Username,VideoID,Comment,Date,Car
0,2024-07-23T15:19:10Z,@raider7966,Las92RDdkOM,Sorry not sure what they were thinking with th...,2024-07-23T15:19:10Z,Mini Cooper
1,2024-07-15T16:47:45Z,@jackdaniels2905,Las92RDdkOM,Heating and cooling controls need to be physic...,2024-07-15T16:47:45Z,Mini Cooper
2,2024-06-20T21:51:48Z,@pqvid,Las92RDdkOM,You know how you can control the regen? With y...,2024-06-20T21:51:48Z,Mini Cooper
3,2024-06-19T19:54:27Z,@silverlover5817,Las92RDdkOM,Thanks for the video. All the videos I have se...,2024-06-19T19:54:27Z,Mini Cooper
4,2024-07-27T11:34:59Z,@Sc0oby26th,Las92RDdkOM,Err.....it's electric,2024-07-27T11:34:59Z,Mini Cooper


In [60]:
# Ensure the timestamp column is in a proper datetime format
# jeep_avenger['Timestamp'] = pd.to_datetime(jeep_avenger['Timestamp'])
# hyundai_kona['Timestamp'] = pd.to_datetime(hyundai_kona['Timestamp'])
# peugeot_e2008['Timestamp'] = pd.to_datetime(peugeot_e2008['Timestamp'])
# vauxhall_mokka['Timestamp'] = pd.to_datetime(vauxhall_mokka['Timestamp'])
# mini_cooper['Timestamp'] = pd.to_datetime(mini_cooper['Timestamp'])

In [61]:
# Merge the datasets
combined_data = pd.concat([jeep_avenger, hyundai_kona, peugeot_e2008,vauxhall_mokka,mini_cooper])

# Sort the combined dataset by timestamp
# combined_data = combined_data.sort_values(by='Timestamp')


In [62]:
combined_data


,Timestamp,Username,VideoID,Comment,Date,Car
0,2024-05-23T11:35:19Z,@SirKelvin,B4pWH-80vRw,You spent more time praising the car like you ...,2024-05-23T11:35:19Z,Jeep Avenger
1,2024-05-18T10:40:45Z,@crowninterior_ldn6971,B4pWH-80vRw,No one talks about how you move the seat. Whet...,2024-05-18T10:40:45Z,Jeep Avenger
2,2024-07-02T21:29:54Z,@sandracampbell4579,B4pWH-80vRw,"It’s manual,and another review states the ster...",2024-07-02T21:31:47Z,Jeep Avenger
3,2024-03-27T17:18:13Z,@Alfacinhadesign,B4pWH-80vRw,Where is the rails for bars? :(,2024-03-27T17:18:13Z,Jeep Avenger
4,2024-03-18T17:11:53Z,@darkobrodski1453,B4pWH-80vRw,Im thinking about buying petrol version of Jee...,2024-03-18T17:11:53Z,Jeep Avenger
...,...,...,...,...,...,...
2174,2024-05-07T22:14:47Z,@stevenjones916,RAtwyLFvmcc,Looks awful.,2024-05-07T22:14:47Z,Mini Cooper
2175,2024-05-07T22:08:25Z,@robnelsonbayvillage,RAtwyLFvmcc,Yellow is the faster color. Love your work. I ...,2024-05-07T22:08:25Z,Mini Cooper
2176,2024-05-07T22:06:19Z,@bill_heywood,RAtwyLFvmcc,Could have stayed in the UK to drive in pourin...,2024-05-07T22:07:49Z,Mini Cooper
2177,2024-05-08T04:56:40Z,@JamesSleeman,RAtwyLFvmcc,"Because you can't fit a camera, and a Manos, ...",2024-05-08T04:56:40Z,Mini Cooper


### Data Cleaning

In [63]:
combined_data.rename(columns={'VideoID': 'ID', 'Comment': 'Text'}, inplace=True)
combined_data.head()

,Timestamp,Username,ID,Text,Date,Car
0,2024-05-23T11:35:19Z,@SirKelvin,B4pWH-80vRw,You spent more time praising the car like you ...,2024-05-23T11:35:19Z,Jeep Avenger
1,2024-05-18T10:40:45Z,@crowninterior_ldn6971,B4pWH-80vRw,No one talks about how you move the seat. Whet...,2024-05-18T10:40:45Z,Jeep Avenger
2,2024-07-02T21:29:54Z,@sandracampbell4579,B4pWH-80vRw,"It’s manual,and another review states the ster...",2024-07-02T21:31:47Z,Jeep Avenger
3,2024-03-27T17:18:13Z,@Alfacinhadesign,B4pWH-80vRw,Where is the rails for bars? :(,2024-03-27T17:18:13Z,Jeep Avenger
4,2024-03-18T17:11:53Z,@darkobrodski1453,B4pWH-80vRw,Im thinking about buying petrol version of Jee...,2024-03-18T17:11:53Z,Jeep Avenger


In [64]:
len(combined_data)

6374

In [65]:
combined_data = combined_data.drop(columns=['Username','ID'])


In [66]:
combined_data.head()

,Timestamp,Text,Date,Car
0,2024-05-23T11:35:19Z,You spent more time praising the car like you ...,2024-05-23T11:35:19Z,Jeep Avenger
1,2024-05-18T10:40:45Z,No one talks about how you move the seat. Whet...,2024-05-18T10:40:45Z,Jeep Avenger
2,2024-07-02T21:29:54Z,"It’s manual,and another review states the ster...",2024-07-02T21:31:47Z,Jeep Avenger
3,2024-03-27T17:18:13Z,Where is the rails for bars? :(,2024-03-27T17:18:13Z,Jeep Avenger
4,2024-03-18T17:11:53Z,Im thinking about buying petrol version of Jee...,2024-03-18T17:11:53Z,Jeep Avenger


In [67]:
# Adjust 'Comment' column to lowercase
combined_data['Text'] = combined_data['Text'].str.lower()
combined_data.head()

,Timestamp,Text,Date,Car
0,2024-05-23T11:35:19Z,you spent more time praising the car like you ...,2024-05-23T11:35:19Z,Jeep Avenger
1,2024-05-18T10:40:45Z,no one talks about how you move the seat. whet...,2024-05-18T10:40:45Z,Jeep Avenger
2,2024-07-02T21:29:54Z,"it’s manual,and another review states the ster...",2024-07-02T21:31:47Z,Jeep Avenger
3,2024-03-27T17:18:13Z,where is the rails for bars? :(,2024-03-27T17:18:13Z,Jeep Avenger
4,2024-03-18T17:11:53Z,im thinking about buying petrol version of jee...,2024-03-18T17:11:53Z,Jeep Avenger


In [68]:
# Remove emojis 
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [69]:
# Remove punctuation 
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

# Apply the functions to the 'Comment' column
combined_data['Text'] = combined_data['Text'].apply(remove_emojis).apply(remove_punctuation)

In [70]:
combined_data.head()

,Timestamp,Text,Date,Car
0,2024-05-23T11:35:19Z,you spent more time praising the car like you ...,2024-05-23T11:35:19Z,Jeep Avenger
1,2024-05-18T10:40:45Z,no one talks about how you move the seat wheth...,2024-05-18T10:40:45Z,Jeep Avenger
2,2024-07-02T21:29:54Z,its manualand another review states the stereo...,2024-07-02T21:31:47Z,Jeep Avenger
3,2024-03-27T17:18:13Z,where is the rails for bars,2024-03-27T17:18:13Z,Jeep Avenger
4,2024-03-18T17:11:53Z,im thinking about buying petrol version of jee...,2024-03-18T17:11:53Z,Jeep Avenger


In [71]:
# Split the 'Timestamp' into 'Date' and 'Time' and keep only the 'Date'
combined_data['Date'] = combined_data['Timestamp'].str.split('T', expand=True)[0]

# Move 'Date' to the first column
cols = ['Date'] + [col for col in combined_data.columns if col != 'Date']
combined_data = combined_data[cols]

# Drop the 'Timestamp' 
combined_data.drop('Timestamp', axis=1, inplace=True)

In [73]:
combined_data.head(15)

,Date,Text,Car
0,2024-05-23,you spent more time praising the car like you ...,Jeep Avenger
1,2024-05-18,no one talks about how you move the seat wheth...,Jeep Avenger
2,2024-07-02,its manualand another review states the stereo...,Jeep Avenger
3,2024-03-27,where is the rails for bars,Jeep Avenger
4,2024-03-18,im thinking about buying petrol version of jee...,Jeep Avenger
5,2024-03-20,i bought petrol,Jeep Avenger
6,2024-05-23,go for petrol youll be missing nothing in ter...,Jeep Avenger
7,2024-03-09,its a mokka stop talking crap,Jeep Avenger
8,2023-12-01,great design utterly ridiculous pricing vehic...,Jeep Avenger
9,2023-11-06,to me it looks like a vw tcross shape with a d...,Jeep Avenger


In [75]:
# Save as a new CSV file
# combined_data.to_csv('competitor_comments_combined_cleaned.csv', index=False)

In [77]:
print(combined_data[combined_data['Text'].isnull()])
print(combined_data[combined_data['Date'].isnull()])

Empty DataFrame
Columns: [Date, Text, Car]
Index: []
Empty DataFrame
Columns: [Date, Text, Car]
Index: []


In [78]:
# Drop rows with any null values
combined_data = combined_data.dropna()

In [79]:
print(combined_data[combined_data['Text'].isnull()])
print(combined_data[combined_data['Date'].isnull()])

Empty DataFrame
Columns: [Date, Text, Car]
Index: []
Empty DataFrame
Columns: [Date, Text, Car]
Index: []


In [80]:
combined_data.to_csv('competitor_comments_combined_cleaned.csv', index=False)